<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/make_z_and_decoded_for_comparison_with_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Download Tensorflow checkpoint
! gdown --id 1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV

%tensorflow_version 1.x

!git clone https://github.com/SourCherries/VAE-GAN-CelebA.git

import sys
sys.path.insert(0,'/content/VAE-GAN-CelebA/libs')
sys.path.insert(0,'/content/VAE-GAN-CelebA/')

try:
  import sys
  import os
  import numpy as np
  import tensorflow as tf
  import matplotlib.pyplot as plt
  import libs.make_network_deterministic as make_network
  import libs.utils as utils
  from scipy.io import loadmat, savemat
except ImportError as e:
  print("Make sure the libs folder is available in current directory.")
  print(e)

  

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV
To: /content/vaegan_celeba.ckpt
100% 555M/555M [00:05<00:00, 103MB/s] 
fatal: destination path 'VAE-GAN-CelebA' already exists and is not an empty directory.


# Encoding

In [4]:
file = "/content/VAE-GAN-CelebA/example.jpg"
print('Input file is ', file)

sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()
if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()

img = plt.imread(file)[..., :3]
img = utils.preprocess128(img,crop_factor=0.8)[np.newaxis]
                
#generate images from z
z = sess.run(Z_mu, feed_dict={X: img, is_training: False})

#save data in Matlab format
savemat(file[:-4]+'_z',dict(latent=z))





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Input file is  /content/VAE-GAN-CelebA/example.jpg
encoder/input: [None, 128, 128, 3]
layer: 0 , shape: [None, 64, 64, 192]
layer: 1 , shape: [None, 32, 32, 256]
layer: 2 , shape: [None, 16, 16, 384]
layer: 3 , shape: [None, 8, 8, 512]


layer: 4 , shape: [None, 4, 4, 768]
Tensor("encoder/variational/random_normal:0", shape=(?, 1024), dtype=float32)



Instructions for updating:
Use `tf.global_variables_initializer` instead.

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


VAE-GAN model restored.


# Decoding

In [5]:
# file = "example_z"
file = "/content/VAE-GAN-CelebA/example_z"

sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()
if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()


#load data from Matlab format
latent = loadmat(file)['latent'] #make sure that the matlab variable name is 'latent'
print('input data to be transformed:',latent.shape)
if latent.shape[0]==1024:
    latent=latent[np.newaxis]
elif latent.shape[1]!=1024:
    print("None of the input dimensions appears to be 1024!!!")
                
#generate images from z
g = sess.run(G, feed_dict={Z: (1*latent), is_training: False})

def imdeprocess(g):
    stretch = 1.0
    for i in range(g.shape[0]):
        g[i]=np.clip(stretch*g[i] / (g.max()),0,1)
    return g

g=imdeprocess(g)
                
#if there's only one image, we save it as a jpg
if g.shape[0]==1:
    plt.imsave(file[:-4]+'_g.jpg',g[0])
                
#in all cases, we save the image(s) as a .mat file
savemat(file[:-4]+'_g',dict(images=g))

encoder/input: [None, 128, 128, 3]
layer: 0 , shape: [None, 64, 64, 192]
layer: 1 , shape: [None, 32, 32, 256]
layer: 2 , shape: [None, 16, 16, 384]
layer: 3 , shape: [None, 8, 8, 512]
layer: 4 , shape: [None, 4, 4, 768]
Tensor("encoder/variational/random_normal:0", shape=(?, 1024), dtype=float32)


VAE-GAN model restored.
input data to be transformed: (1, 1024)
